In [ ]:
from src.util.router import Router
from testdata.email import EMAIL

router = Router()
output = router.route_input(EMAIL)
print(output)


In [ ]:
from typing import Literal

from gen_ai_hub.proxy.langchain import init_llm
from pydantic import BaseModel, Field
from testdata.email import EMAIL


class RouterOutputModel(BaseModel):
    """Analyze the unread email and route it according to its content."""

    reasoning: str = Field(...,
        description="Step-by-step reasoning behind the classification."
    )
    classification: Literal["ignore", "respond", "notify"] = Field(...,
        description="The classification of an email: 'ignore' for irrelevant emails, "
        "'notify' for important information that doesn't need a response, "
        "'respond' for emails that need a reply",
    )

class SearchQuery(BaseModel):
    search_query: str = Field(..., description="Query that is optimized web search.")
    justification: str = Field(
        ..., description="Why this query is relevant to the user's request."
    )

llm = init_llm("gpt-4o")
if llm is None:
    raise ValueError("LLM initialization failed! Check API key or `gen_ai_hub` installation.")

print(f"LLM initialized: {llm}")

structured_llm = llm.with_structured_output(SearchQuery)

print(f"Structured LLM: {structured_llm}")

email_text = EMAIL.get("body", "No email content provided.")
print(f"Email Text: {email_text}")  # Debugging

prompt = f"Analyze the following email and classify it:\n\n{email_text}"

try:
    output = structured_llm.invoke(prompt)
    print(f"Model Output: {output}")
except Exception as e:
    print(f"Error invoking structured_llm: {e}")



In [1]:
from pydantic import BaseModel, Field
from gen_ai_hub.proxy.langchain import init_llm
from gen_ai_hub.proxy.langchain.openai import ChatOpenAI

#llm = ChatOpenAI(proxy_model_name='gpt-4o', temperature=0)
llm = init_llm('gemini-1.5-pro', max_tokens=1024)
print(llm.invoke("Are you there?").content)

class SearchQuery(BaseModel):
    search_query: str = Field(..., description="Query that is optimized web search.")
    justification: str = Field(
        ..., description="Why this query is relevant to the user's request."
    )

# Augment the LLM with schema for structured output
structured_llm = llm.with_structured_output(SearchQuery)

# Invoke the augmented LLM
print(structured_llm.invoke("How does Calcium CT score relate to high cholesterol?").content)

ValueError: No deployment found with: deployment.model_name == gemini-1.5-pro